In [1]:
from glob import glob
import json
import re
import yaml
import os
import shutil
import time
os.chdir('../')

In [2]:
with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

In [3]:
queues_cfg = cfg['QUEUES']

In [4]:
tweets_queue_path = queues_cfg['new_tweets']
processed_tweets_path = queues_cfg['processed_tweets']
sa_queue_path = queues_cfg['sentiment_tasks']
geo_queue_path = queues_cfg['geo_tasks']

In [5]:
#for folder in [tweets_folder, sentiment_folder, geo_folder, 'tweet_no_coordinates']:
for folder in [tweets_queue_path, processed_tweets_path, ]:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [6]:
unprocessed_tweets = glob('{}/*.json'.format(tweets_queue_path))
len(unprocessed_tweets)

358

# Process Downloaded Tweets Queue

In [7]:
def create_geoanalyser_task(tweet_id, coordinates):
    filename = '{}/{}.task.txt'.format(geo_queue_path, tweet_id)
    with open(filename, 'w') as fp:
        fp.write(json.dumps(coordinates))
        fp.close()

def send_to_geoanalyser(tweet_id, tweet_json):
    try:
        if tweet_json['coordinates'] is not None:
            coordinates = tweet_json['coordinates']
        elif tweet_json['place'] is not None:
            coordinates = tweet_json['place']['bounding_box']
        else:
            print('Tweet {} doesn\'t contain coordinates'.format(tweet_id))
            print(path)
        create_geoanalyser_task(tweet_id, coordinates=coordinates)
    except Exception as e:
        print('Tweet {} wasn\'t sent to geoanalyser due to error. {}'.format(tweet_id, e))
        #raise e

In [8]:
def create_sentiment_task(tweet_id, text):
    filename = '{}/{}.task.txt'.format(sa_queue_path, tweet_id)
    with open(filename, 'w') as fp:
        fp.write(text)
        fp.close()
        
def send_to_sentiment_analysis(tweet_id, tweet_json):
    try:
        if tweet_json['text'] is not None:
            text = tweet_json['text']
        else:
            print('Tweet {} doesn\'t contain text'.format(tweet_id))
            print(path)
        create_sentiment_task(tweet_id, text)
    except Exception as e:
        print('Tweet {} wasn\'t sent to sentiment analyser due to error. {}'.format(tweet_id, e))

In [9]:
def save_to_couch_db(path):
    filename = path.split('/')[-1]
    new_path = '{}/{}'.format(processed_tweets_path, filename)
    shutil.move(path, new_path)

In [10]:
!cp $processed_tweets_path/* $tweets_queue_path

/bin/sh: 1: cp: Argument list too long


In [ ]:
#for path in bitcoin_pathes:
i = 1
while True:
    unprocessed_tweets = glob('{}/*.json'.format(tweets_queue_path))
    for path in unprocessed_tweets:
        with open(path, 'r') as fp:
            tweet_json = json.load(fp)
            tweet_id = tweet_json['id_str']
            for analyser_func in (send_to_geoanalyser, send_to_sentiment_analysis):
                analyser_func(tweet_id, tweet_json)
            save_to_couch_db(path)
    
    print('Iteration: {}\tFiles processed: {}'.format(i, len(unprocessed_tweets)))
    i+=1
    time.sleep(20)

Iteration: 1	Files processed: 358
Iteration: 2	Files processed: 13
Iteration: 3	Files processed: 9
Iteration: 4	Files processed: 4
Iteration: 5	Files processed: 13
Iteration: 6	Files processed: 10
Iteration: 7	Files processed: 9
Iteration: 8	Files processed: 16
Iteration: 9	Files processed: 5
Iteration: 10	Files processed: 13
Iteration: 11	Files processed: 9
Iteration: 12	Files processed: 8
Iteration: 13	Files processed: 13
Iteration: 14	Files processed: 13
Iteration: 15	Files processed: 12
Iteration: 16	Files processed: 10
Iteration: 17	Files processed: 10
Iteration: 18	Files processed: 6
Iteration: 19	Files processed: 7
Iteration: 20	Files processed: 13
Iteration: 21	Files processed: 7
Iteration: 22	Files processed: 4
Iteration: 23	Files processed: 14
Iteration: 24	Files processed: 6
Iteration: 25	Files processed: 11
Iteration: 26	Files processed: 12
Iteration: 27	Files processed: 7
Iteration: 28	Files processed: 5
Iteration: 29	Files processed: 9
Iteration: 30	Files processed: 15
It

Iteration: 243	Files processed: 6
Iteration: 244	Files processed: 3
Iteration: 245	Files processed: 7
Iteration: 246	Files processed: 14
Iteration: 247	Files processed: 10
Iteration: 248	Files processed: 7
Iteration: 249	Files processed: 6
Iteration: 250	Files processed: 7
Iteration: 251	Files processed: 8
Iteration: 252	Files processed: 9
Iteration: 253	Files processed: 9
Iteration: 254	Files processed: 8
Iteration: 255	Files processed: 11
Iteration: 256	Files processed: 6
Iteration: 257	Files processed: 9
Iteration: 258	Files processed: 10
Iteration: 259	Files processed: 5
Iteration: 260	Files processed: 7
Iteration: 261	Files processed: 8
Iteration: 262	Files processed: 10
Iteration: 263	Files processed: 14
Iteration: 264	Files processed: 8
Iteration: 265	Files processed: 7
Iteration: 266	Files processed: 10
Iteration: 267	Files processed: 15
Iteration: 268	Files processed: 8
Iteration: 269	Files processed: 6
Iteration: 270	Files processed: 6
Iteration: 271	Files processed: 12
Itera

Iteration: 484	Files processed: 1
Iteration: 485	Files processed: 5
Iteration: 486	Files processed: 3
Iteration: 487	Files processed: 5
Iteration: 488	Files processed: 3
Iteration: 489	Files processed: 4
Iteration: 490	Files processed: 4
Iteration: 491	Files processed: 2
Iteration: 492	Files processed: 4
Iteration: 493	Files processed: 4
Iteration: 494	Files processed: 2
Iteration: 495	Files processed: 3
Iteration: 496	Files processed: 7
Iteration: 497	Files processed: 3
Iteration: 498	Files processed: 0
Iteration: 499	Files processed: 3
Iteration: 500	Files processed: 2
Iteration: 501	Files processed: 4
Iteration: 502	Files processed: 2
Iteration: 503	Files processed: 4
Iteration: 504	Files processed: 3
Iteration: 505	Files processed: 1
Iteration: 506	Files processed: 2
Iteration: 507	Files processed: 3
Iteration: 508	Files processed: 3
Iteration: 509	Files processed: 2
Iteration: 510	Files processed: 3
Iteration: 511	Files processed: 5
Iteration: 512	Files processed: 1
Iteration: 513

Iteration: 778	Files processed: 3
Iteration: 779	Files processed: 0
Iteration: 780	Files processed: 4
Iteration: 781	Files processed: 3
Iteration: 782	Files processed: 1
Iteration: 783	Files processed: 2
Iteration: 784	Files processed: 1
Iteration: 785	Files processed: 0
Iteration: 786	Files processed: 3
Iteration: 787	Files processed: 0
Iteration: 788	Files processed: 2
Iteration: 789	Files processed: 7
Iteration: 790	Files processed: 1
Iteration: 791	Files processed: 3
Iteration: 792	Files processed: 2
Iteration: 793	Files processed: 1
Iteration: 794	Files processed: 2
Iteration: 795	Files processed: 1
Iteration: 796	Files processed: 3
Iteration: 797	Files processed: 1
Iteration: 798	Files processed: 4
Iteration: 799	Files processed: 4
Iteration: 800	Files processed: 0
Iteration: 801	Files processed: 5
Iteration: 802	Files processed: 4
Iteration: 803	Files processed: 1
Iteration: 804	Files processed: 3
Iteration: 805	Files processed: 1
Iteration: 806	Files processed: 0
Iteration: 807

Iteration: 1019	Files processed: 2
Iteration: 1020	Files processed: 0
Iteration: 1021	Files processed: 0
Iteration: 1022	Files processed: 1
Iteration: 1023	Files processed: 1
Iteration: 1024	Files processed: 0
Iteration: 1025	Files processed: 0
Iteration: 1026	Files processed: 0
Iteration: 1027	Files processed: 0
Iteration: 1028	Files processed: 2
Iteration: 1029	Files processed: 1
Iteration: 1030	Files processed: 2
Iteration: 1031	Files processed: 1
Iteration: 1032	Files processed: 0
Iteration: 1033	Files processed: 0
Iteration: 1034	Files processed: 0
Iteration: 1035	Files processed: 1
Iteration: 1036	Files processed: 1
Iteration: 1037	Files processed: 1
Iteration: 1038	Files processed: 3
Iteration: 1039	Files processed: 1
Iteration: 1040	Files processed: 2
Iteration: 1041	Files processed: 1
Iteration: 1042	Files processed: 0
Iteration: 1043	Files processed: 1
Iteration: 1044	Files processed: 0
Iteration: 1045	Files processed: 0
Iteration: 1046	Files processed: 2
Iteration: 1047	File

Iteration: 1254	Files processed: 0
Iteration: 1255	Files processed: 0
Iteration: 1256	Files processed: 1
Iteration: 1257	Files processed: 0
Iteration: 1258	Files processed: 0
Iteration: 1259	Files processed: 1
Iteration: 1260	Files processed: 0
Iteration: 1261	Files processed: 1
Iteration: 1262	Files processed: 1
Iteration: 1263	Files processed: 0
Iteration: 1264	Files processed: 3
Iteration: 1265	Files processed: 0
Iteration: 1266	Files processed: 1
Iteration: 1267	Files processed: 0
Iteration: 1268	Files processed: 1
Iteration: 1269	Files processed: 1
Iteration: 1270	Files processed: 0
Iteration: 1271	Files processed: 1
Iteration: 1272	Files processed: 3
Iteration: 1273	Files processed: 3
Iteration: 1274	Files processed: 2
Iteration: 1275	Files processed: 3
Iteration: 1276	Files processed: 1
Iteration: 1277	Files processed: 0
Iteration: 1278	Files processed: 1
Iteration: 1279	Files processed: 0
Iteration: 1280	Files processed: 0
Iteration: 1281	Files processed: 1
Iteration: 1282	File

Iteration: 1489	Files processed: 1
Iteration: 1490	Files processed: 3
Iteration: 1491	Files processed: 1
Iteration: 1492	Files processed: 2
Iteration: 1493	Files processed: 1
Iteration: 1494	Files processed: 0
Iteration: 1495	Files processed: 3
Iteration: 1496	Files processed: 5
Iteration: 1497	Files processed: 0
Iteration: 1498	Files processed: 1
Iteration: 1499	Files processed: 2
Iteration: 1500	Files processed: 2
Iteration: 1501	Files processed: 4
Iteration: 1502	Files processed: 0
Iteration: 1503	Files processed: 1
Iteration: 1504	Files processed: 4
Iteration: 1505	Files processed: 1
Iteration: 1506	Files processed: 4
Iteration: 1507	Files processed: 2
Iteration: 1508	Files processed: 2
Iteration: 1509	Files processed: 2
Iteration: 1510	Files processed: 1
Iteration: 1511	Files processed: 3
Iteration: 1512	Files processed: 5
Iteration: 1513	Files processed: 0


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## test

In [15]:
with open('../shared_folder/tweets/988986510979497985.json', 'r') as fp:
    data = json.load(fp)   
#data

# Analyse tweets contents

In [8]:
hashtags = ['bitcoin', 'blockchain', 'btc', 'cryptocurrency', 'market\svalue',
            'crypto','ethereum', 'fintech', 'coin', 'doge', 'ethereum', 'ripple', 'litecoin', 
            'cardano', 'monero', 'TRON', 'zcash', 'jaxx', 'copay', 'bitpay', 'exodus', 
            'mycelium', 'Bread\sWallet', 'trezor', 'ledger\snano', 'Silk\sRoad', 'darknet\smarket', 
            'dogecoin', 'ASIC\sMiner', 'Central\sLedger', 'Hashrate', 'ICO',         
           ]

In [9]:
'|'.join(hashtags)

'bitcoin|blockchain|btc|cryptocurrency|market\\svalue|crypto|ethereum|fintech|coin|doge|ethereum|ripple|litecoin|cardano|monero|TRON|zcash|jaxx|copay|bitpay|exodus|mycelium|Bread\\sWallet|trezor|ledger\\snano|Silk\\sRoad|darknet\\smarket|dogecoin|ASIC\\sMiner|Central\\sLedger|Hashrate|ICO'

In [10]:
bitcoin_pathes = []
for path in (unprocessed_tweets):
    with open(path, 'r') as fp:
        data = json.load(fp)
        if re.search('|'.join(hashtags), json.dumps(data)):
            bitcoin_pathes.append(path)
        #print(data['entities']['hashtags'])


In [ ]:
len(bitcoin_pathes)